In [0]:
import copy
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import datasets, transforms

In [0]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [0]:
train_transform = transforms.Compose([
  transforms.Resize(224),
  transforms.RandomChoice((transforms.RandomHorizontalFlip(), transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3), transforms.RandomVerticalFlip(), transforms.RandomRotation(0.2))),
  transforms.ToTensor(),
  transforms.Normalize(mean = mean, std = std)                                      
]) 

In [0]:
test_transform = transforms.Compose([
  transforms.Resize(224),
  transforms.ToTensor(),
  transforms.Normalize(mean = mean, std = std)                                      
])

In [0]:
# !unzip '/content/drive/My Drive/Deep_Learning_Projects/Currency_Dataset/CurrencyDataset.zip' -d '/content/drive/My Drive/Deep_Learning_Projects/Currency_Dataset/'

In [0]:
train_dir = "/content/drive/My Drive/Deep_Learning_Projects/Currency_Dataset/CurrencyDataset/train-cropped"
test_dir = "/content/drive/My Drive/Deep_Learning_Projects/Currency_Dataset/CurrencyDataset/test-cropped"
valid_dir = "/content/drive/My Drive/Deep_Learning_Projects/Currency_Dataset/CurrencyDataset/valid-cropped"
batch_size = 16
num_workers = 2

In [0]:
cd '/content/drive/My Drive/Deep_Learning_Projects/Currency_Dataset/CurrencyDataset'

/content/drive/My Drive/Deep_Learning_Projects/Currency_Dataset/CurrencyDataset


In [0]:
train_set = datasets.ImageFolder("train-cropped", transform = train_transform)

In [0]:
train_set.classes

['10', '100', '20', '200', '2000', '5', '50', '500', 'none']

In [0]:
type(train_set)

torchvision.datasets.folder.ImageFolder

In [0]:
val_set = datasets.ImageFolder("valid-cropped", transform = test_transform)

In [0]:
val_set

Dataset ImageFolder
    Number of datapoints: 871
    Root location: valid-cropped
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485 0.456 0.406], std=[0.229 0.224 0.225])
           )

In [0]:
val_set.classes

['10', '100', '20', '200', '2000', '5', '50', '500', 'none']

In [0]:
test_set = datasets.ImageFolder("test-cropped", transform = test_transform)

In [0]:
test_set

Dataset ImageFolder
    Number of datapoints: 294
    Root location: test-cropped
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485 0.456 0.406], std=[0.229 0.224 0.225])
           )

In [0]:
trainloader = torch.utils.data.DataLoader(train_set,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          num_workers = num_workers)

In [0]:
val_loader = torch.utils.data.DataLoader(val_set,
                                          batch_size = batch_size,
                                          shuffle = False,
                                          num_workers = num_workers)

In [0]:
testloader = torch.utils.data.DataLoader(test_set,
                                          batch_size = batch_size,
                                          shuffle = False,
                                          num_workers = num_workers)

In [0]:
dataloaders = {
    'train' : trainloader,
    'val' : val_loader
} 

In [0]:
total_batch_sizes = {'train' : len(trainloader), 'val' : len(val_loader)}

In [0]:
dataset_sizes = {'train' : len(trainloader), 'val' : len(val_loader)}

In [0]:
dataset_sizes

{'train': 276, 'val': 55}

In [0]:
class_names = train_set.classes

print(np.size(class_names))
print(class_names)

9
['10', '100', '20', '200', '2000', '5', '50', '500', 'none']


#Import Model

In [0]:
from torchvision import models

In [0]:
model = models.resnet101(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/checkpoints/resnet101-5d3b4d8f.pth


In [0]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Freezing model parameters of above model

In [0]:
for param in model.parameters():
  param.requires_grad = False

In [0]:
num_ftrs = model.fc.in_features
num_ftrs

2048

In [0]:
model.fc  = nn.Sequential(nn.Linear(num_ftrs, 1024),
                           nn.ReLU(),
                           nn.Linear(1024, 512),
                           nn.ReLU(),
                           nn.Linear(512, 9),
                           nn.LogSoftmax(dim=1))

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
print(device)

cuda:0


In [0]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [0]:
criterion_ft = nn.NLLLoss()

optimizer_ft = optim.Adam(model.parameters())

In [0]:
def train_model(model, criterion, optimizer, num_epochs=2):

  model = model.to(device)

  for epoch in range(num_epochs):
    train_loss = 0
    val_loss = 0
    accuracy = 0

    #Training model
    model.train()
    counter = 0
    for inputs, labels in trainloader:
      inputs = inputs.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()

      output = model.forward(inputs)

      loss = criterion(output, labels)

      loss.backward()

      optimizer.step()

      train_loss += loss.item()*inputs.size(0)

      #printing progress of training
      counter += 1
      if(counter % 5 == 0):
        print(counter, "/", len(trainloader))  
      
    #Evaluating Model
    model.eval()
    counter = 0
    with torch.no_grad():
      total = 0
      correct = 0
      for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)  

        output = model.forward(inputs)

        valloss = criterion(output, labels)

        val_loss += valloss.item()*inputs.size(0)

        _, predicted = torch.max(output.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

      train_loss = train_loss/len(trainloader.dataset)
      valid_loss = val_loss/len(val_loader.dataset)
      # Print out the information
      print("Accuracy of the model on test images: {}%".format(100*correct/total))
      print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, val_loss))      

In [0]:
train_model(model, criterion_ft, optimizer_ft, num_epochs=4)

5 / 276
10 / 276
15 / 276
20 / 276
25 / 276
30 / 276
35 / 276
40 / 276
45 / 276
50 / 276
55 / 276
60 / 276
65 / 276
70 / 276
75 / 276
80 / 276
85 / 276
90 / 276
95 / 276
100 / 276
105 / 276
110 / 276
115 / 276
120 / 276
125 / 276
130 / 276
135 / 276
140 / 276
145 / 276
150 / 276
155 / 276
160 / 276
165 / 276
170 / 276
175 / 276
180 / 276
185 / 276
190 / 276
195 / 276
200 / 276
205 / 276
210 / 276
215 / 276
220 / 276
225 / 276
230 / 276
235 / 276
240 / 276
245 / 276
250 / 276
255 / 276
260 / 276
265 / 276
270 / 276
275 / 276
Accuracy of the model on test images: 78.300803673938%
Epoch: 0 	Training Loss: 1.259651 	Validation Loss: 612.548595
5 / 276
10 / 276
15 / 276
20 / 276
25 / 276
30 / 276
35 / 276
40 / 276
45 / 276
50 / 276
55 / 276
60 / 276
65 / 276
70 / 276
75 / 276
80 / 276
85 / 276
90 / 276
95 / 276
100 / 276
105 / 276
110 / 276
115 / 276
120 / 276
125 / 276
130 / 276
135 / 276
140 / 276
145 / 276
150 / 276
155 / 276
160 / 276
165 / 276
170 / 276
175 / 276
180 / 276
185 / 276
19

In [0]:
train_model(model, criterion_ft, optimizer_ft, num_epochs=20)

5 / 276
10 / 276
15 / 276
20 / 276
25 / 276
30 / 276
35 / 276
40 / 276
45 / 276
50 / 276
55 / 276
60 / 276
65 / 276
70 / 276
75 / 276
80 / 276
85 / 276
90 / 276
95 / 276
100 / 276
105 / 276
110 / 276
115 / 276
120 / 276
125 / 276
130 / 276
135 / 276
140 / 276
145 / 276
150 / 276
155 / 276
160 / 276
165 / 276
170 / 276
175 / 276
180 / 276
185 / 276
190 / 276
195 / 276
200 / 276
205 / 276
210 / 276
215 / 276
220 / 276
225 / 276
230 / 276
235 / 276
240 / 276
245 / 276
250 / 276
255 / 276
260 / 276
265 / 276
270 / 276
275 / 276
Accuracy of the model on test images: 87.0264064293915%
Epoch: 0 	Training Loss: 0.622135 	Validation Loss: 346.887070
5 / 276
10 / 276
15 / 276
20 / 276
25 / 276
30 / 276
35 / 276
40 / 276
45 / 276
50 / 276
55 / 276
60 / 276
65 / 276
70 / 276
75 / 276
80 / 276
85 / 276
90 / 276
95 / 276
100 / 276
105 / 276
110 / 276
115 / 276
120 / 276
125 / 276
130 / 276
135 / 276
140 / 276
145 / 276
150 / 276
155 / 276
160 / 276
165 / 276
170 / 276
175 / 276
180 / 276
185 / 276
1